# per Tissue Analysis and Models
*Ben Imlay*

## Database Setup

In [19]:
import pandas as pd
from pathlib import Path
from os import listdir
TISSUE='Liver'

In [ ]:
manifest={"data":"All_Tissue_Site_Details.combined.reads.gct",
              "sample_meta":"GTEx_v7_Annotations_SampleAttributesDS.txt",
              "subject_meta":"GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
               "merged_meta":"merged_meta.tsv"}
meta=pd.read_csv(data_dir/manifest['merged_meta'],sep="\t",dtype={'SMUBRID':object,'SEX':object,'DTHHRDY':object})

data_dir=Path("data")
infiles=listdir(data_dir/"tissue-specific")

meta.iloc[0:3,:]

## Avaiable Tissues

In [23]:
# To do python plots of counts
TISSUE_files=[f for f in infiles if  TISSUE in f]
TISSUE_files

['Liver_lcpm.tsv', 'Liver_cpm.tsv']

In [26]:
meta['AGE'].unique()

array(['60-69', '50-59', '40-49', '20-29', '30-39', '70-79', nan],
      dtype=object)